In [1]:
!pip install transformers

In [2]:
import re
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import transformers
from transformers import BertTokenizer
from transformers import TFAutoModel

In [3]:
train = pd.read_csv('/kaggle/input/testtrainsample/train.csv')
test = pd.read_csv('/kaggle/input/testtrainsample/test.csv')

In [4]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train["length"] = train["text"].apply(lambda x : len(x))
test["length"] = test["text"].apply(lambda x : len(x))

In [6]:
train["length"].describe()

count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64

In [7]:
train['text'] = train['text'].apply(lambda x: " ".join([word.lower() for word in str(x).split()]))
test['text'] = test['text'].apply(lambda x: " ".join([word.lower() for word in str(x).split()]))

In [8]:
import re
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)

    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)

    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text

def clean(tweet):

    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)

    Special = '@#!?+&*[]-%:/()$=><|{}^'
    for s in Special:
        tweet = tweet.replace(s, "")

    return tweet

train['text'] = train['text'].apply(lambda s : clean(s))
test['text'] = test['text'].apply(lambda s : clean(s))
train['text'] = train['text'].apply(lambda s : remove_html(s))
test['text'] = test['text'].apply(lambda s : remove_html(s))
train['text'] = train['text'].apply(lambda s : remove_emoji(s))
test['text'] = test['text'].apply(lambda s : remove_emoji(s))
train['text'] = train['text'].apply(lambda s : decontraction(s))
test['text'] = test['text'].apply(lambda s : decontraction(s))

In [9]:
train.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,69
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38
2,5,NaN,NaN,all residents asked to ishelter in place' are...,1,133
3,6,NaN,NaN,"13,000 people receive wildfires evacuation ord...",1,65
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,88


In [10]:
import tensorflow as tf
seq_len = 256
batch_size = 40
num_samples = len(train)
model_name = 'cardiffnlp/twitter-roberta-base-sentiment'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

train_tokens = tokenizer(
    train['text'].tolist(),
    max_length=seq_len,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='np'
)
#one hot encoding
y_train = train['target'].values
labels = np.zeros((num_samples, y_train.max() + 1))
labels[np.arange(num_samples), y_train] = 1

dataset = tf.data.Dataset.from_tensor_slices(
    (
        train_tokens['input_ids'],
        train_tokens['attention_mask'],
        labels
    )
)

def map_func(input_ids, masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }, labels

dataset = dataset.map(map_func)
dataset = dataset.shuffle(10000).batch(batch_size=batch_size, drop_remainder=True)

split = 0.7
size = int((train_tokens['input_ids'].shape[0] // batch_size) * split)

train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
model = TFAutoModel.from_pretrained(model_name)


input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')


embeddings = model(input_ids, attention_mask=mask)[0]
embeddings = embeddings[:, 0, :]

x = tf.keras.layers.Dense(512, activation='relu')(embeddings)

y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)



optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = bert_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8,
    batch_size=30
)

Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/8
133/133 [==============================] - 216s 1s/step - loss: 0.4789 - binary_accuracy: 0.7729 - val_loss: 0.3458 - val_binary_accuracy: 0.8623
Epoch 2/8
133/133 [==============================] - 175s 1s/step - loss: 0.3844 - binary_accuracy: 0.8383 - val_loss: 0.3193 - val_binary_accuracy: 0.8724
Epoch 3/8
133/133 [==============================] - 175s 1s/step - loss: 0.3256 - binary_accuracy: 0.8660 - val_loss: 0.2911 - val_binary_accuracy: 0.8820
Epoch 4/8
133/133 [==============================] - 175s 1s/step - loss: 0.2979 - binary_accuracy: 0.8831 - val_loss: 0.2272 - val_binary_accuracy: 0.9167
Epoch 5/8


In [15]:
bert_model.evaluate(val_ds)

143/143 [==============================] - 39s 275ms/step - loss: 0.0482 - binary_accuracy: 0.9799


[0.0481700673699379, 0.9798951148986816]

In [16]:
def prep_data(text):
    tokens = tokenizer(
        text, max_length=256, truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask']
    }

test['target'] = None

for i, row in test.iterrows():
    tokens = prep_data(row['text'])
    probs = bert_model.predict_on_batch(tokens)
    pred = np.argmax(probs)
    test.at[i, 'target'] = pred

test['target'] = test['target'].astype(int)

In [17]:
test.head()

,id,keyword,location,text,length,target
0,0,NaN,NaN,just happened a terrible car crash,34,1
1,2,NaN,NaN,"heard about earthquake is different cities, st...",64,1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,1
3,9,NaN,NaN,apocalypse lighting. spokane wildfires,40,1
4,11,NaN,NaN,typhoon soudelor kills 28 in china and taiwan,45,1
